In [4]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import xgboost
import sklearn.cluster
import tensorflow as tf

from geoband import API

In [3]:
input_path = pathlib.Path('./data')
if not input_path.is_dir():
    input_path.mkdir()

In [4]:
API.GetCompasData('SBJ_2307_003', '1', input_path.joinpath('1.김해시성연령별_유동인구.csv'))
API.GetCompasData('SBJ_2307_003', '2', input_path.joinpath('2.김해시시간대별_유동인구.csv'))
API.GetCompasData('SBJ_2307_003', '3', input_path.joinpath('3.김해시요일별_유동인구.csv'))
API.GetCompasData('SBJ_2307_003', '4', input_path.joinpath('4.김해시공공wifi_위치정보.csv'))
API.GetCompasData('SBJ_2307_003', '5', input_path.joinpath('5.김해시공공wifi_사용량.csv'))
API.GetCompasData('SBJ_2307_003', '6', input_path.joinpath('6.김해시_CCTV설치현황.csv'))
API.GetCompasData('SBJ_2307_003', '7', input_path.joinpath('7.김해시안전비상벨설치현황.csv'))
API.GetCompasData('SBJ_2307_003', '8', input_path.joinpath('8.김해시보안등설치현황.csv'))
API.GetCompasData('SBJ_2307_003', '9', input_path.joinpath('9.김해시신호등설치현황.csv'))
API.GetCompasData('SBJ_2307_003', '10', input_path.joinpath('10.김해시상세도로망.geojson'))
API.GetCompasData('SBJ_2307_003', '11', input_path.joinpath('11.김해시실폭도로.geojson'))
API.GetCompasData('SBJ_2307_003', '12', input_path.joinpath('12.김해시버스정류장_위치정보.csv'))
API.GetCompasData('SBJ_2307_003', '13', input_path.joinpath('13.김해시버스정류장_승하차이력.csv'))
API.GetCompasData('SBJ_2307_003', '14', input_path.joinpath('14.김해시경전철역현황.csv'))
API.GetCompasData('SBJ_2307_003', '15', input_path.joinpath('15.김해시경전철역별_승하차이력.csv'))
API.GetCompasData('SBJ_2307_003', '16', input_path.joinpath('16.김해시버스노선별_wifi사용량.csv'))
API.GetCompasData('SBJ_2307_003', '17', input_path.joinpath('17.김해시공원현황.csv'))
API.GetCompasData('SBJ_2307_003', '18', input_path.joinpath('18.김해시_주변관광지정보(문화재).geojson'))
API.GetCompasData('SBJ_2307_003', '19', input_path.joinpath('19.김해시_상권정보.csv'))
API.GetCompasData('SBJ_2307_003', '20', input_path.joinpath('20.김해시_법정경계.geojson'))
API.GetCompasData('SBJ_2307_003', '21', input_path.joinpath('21.김해시_행정경계.geojson'))
API.GetCompasData('SBJ_2307_003', '22', input_path.joinpath('22.김해시_격자(매핑용).geojson'))
API.GetCompasData('SBJ_2307_003', '23', input_path.joinpath('23.김해시_토지소유정보.geojson'))
API.GetCompasData('SBJ_2307_003', '24', input_path.joinpath('24.김해시_연속지적도.geojson'))
API.GetCompasData('SBJ_2307_003', '25', input_path.joinpath('25.김해시_공공wifi설치버스정류장별_Wifi사용량(NIA).csv'))

for path in list(input_path.glob('.csv')) + list(input_path.glob('.geojson')):
    print(path)

[2023-09-07 20:26:39] 성공 - data/1.김해시성연령별_유동인구.csv
[2023-09-07 20:26:46] 성공 - data/2.김해시시간대별_유동인구.csv
[2023-09-07 20:26:50] 성공 - data/3.김해시요일별_유동인구.csv
[2023-09-07 20:26:50] 성공 - data/4.김해시공공wifi_위치정보.csv
[2023-09-07 20:26:50] 성공 - data/5.김해시공공wifi_사용량.csv
[2023-09-07 20:26:50] 성공 - data/6.김해시_CCTV설치현황.csv
[2023-09-07 20:26:50] 성공 - data/7.김해시안전비상벨설치현황.csv
[2023-09-07 20:26:50] 성공 - data/8.김해시보안등설치현황.csv
[2023-09-07 20:26:50] 성공 - data/9.김해시신호등설치현황.csv
[2023-09-07 20:26:51] 성공 - data/10.김해시상세도로망.geojson
[2023-09-07 20:26:51] 성공 - data/11.김해시실폭도로.geojson
[2023-09-07 20:26:51] 성공 - data/12.김해시버스정류장_위치정보.csv
[2023-09-07 20:26:51] 성공 - data/13.김해시버스정류장_승하차이력.csv
[2023-09-07 20:26:51] 성공 - data/14.김해시경전철역현황.csv
[2023-09-07 20:26:51] 성공 - data/15.김해시경전철역별_승하차이력.csv
[2023-09-07 20:26:51] 성공 - data/16.김해시버스노선별_wifi사용량.csv
[2023-09-07 20:26:51] 성공 - data/17.김해시공원현황.csv
[2023-09-07 20:26:51] 성공 - data/18.김해시_주변관광지정보(문화재).geojson
[2023-09-07 20:26:52] 성공 - data/19.김해시_상권정보.csv
[2023-09-07 20:26:5

# 버스

In [6]:
Bus_Routewise_WiFi_Usage_Data=pd.read_csv(input_path.joinpath('16.김해시버스노선별_wifi사용량.csv'))

In [7]:
Bus_Routewise_WiFi_Usage_Data

,line_no,std_ym,usage
0,1,202201,439.96
1,1-1,202201,521.63
2,2,202201,136.94
3,2-1,202201,146.37
4,3,202201,27.21
...,...,...,...
1039,상동공영1,202306,34.83
1040,상동공영2,202306,16.14
1041,예비,202306,158.04
1042,예비(삼),202306,98.39


버스 정류장별 위도 경도와 버스 노선 데이터를 어떻게 합치지?
- Urban_Bus_Operation_Status 에서 경유지 부분 - 기준으로 다 나누기

In [ ]:
Bus_Location=pd.read_csv(input_path.joinpath('12.김해시버스정류장_위치정보.csv'))
Bus_Location.T

In [8]:
Urban_Bus_Operation_Status=pd.read_csv('Kimhae Urban Bus Operation Status.csv',encoding='euc-kr')
Urban_Bus_Operation_Status

,업체구분,노선번호,기점,종점,첫차,막차,배차간격,운행대수,운행거리(키로미터),경유지
0,가야IBS,1번,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
1,가야IBS,1-1번,삼계기점,김해대학역,05:08/05:30,22:30/23:20,10분~15분,10.0,15.5,북부동사무소-푸르지오2차가락로-한일여고-안동4
2,동부교통,2번,외동차고지,어방동,05:30/-,22:15/-,24~26분,5.0,27.6,가야고-금강병원-시청-안동6-인제대
3,동부교통,2-1번,외동차고지,어방동,05:40/-,22:30/-,24~26분,5.0,27.0,가야고-세무서-시청-안동공단입구-인제대
4,김해버스,3번,풍유동차고지,김해여객터미널(순환),06:10/-,21:20/-,고정배차,2.0,30.4,골든루트-마찰-흥동3-동아그린-가락로-교육청
...,...,...,...,...,...,...,...,...,...,...
66,부산버스,128-1번,구산동,신라대학교,04:40/-,21:50/-,5~7분,26.0,54.0,구산동-인제대학교-신어중학교-구포역-신라대학교
67,부산버스,221번,장유온천,하단전철역,06:00/-,20:00/-,-,NaN,NaN,장유온천-한림풀에버-수가마을-렛츠런파크입구-하단역
68,부산버스,1004번,구산동,국제여객터미널,04:50/-,21:40/-,11~13분,NaN,NaN,구산동-금강병원-불암역-화인아파트-서면역-부산항국제여객터미널
69,부산버스,1004(심야)번,구산동,서구청,22:00/-,23:40/-,30분,NaN,NaN,구산동-금강병원-불암역-북부면허시험장-사상우체국-범일역-남포동


In [9]:
Urban_Bus_Operation_Status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   업체구분        71 non-null     object 
 1   노선번호        71 non-null     object 
 2   기점          71 non-null     object 
 3   종점          71 non-null     object 
 4   첫차          71 non-null     object 
 5   막차          71 non-null     object 
 6   배차간격        71 non-null     object 
 7   운행대수        59 non-null     float64
 8   운행거리(키로미터)  53 non-null     float64
 9   경유지         71 non-null     object 
dtypes: float64(2), object(8)
memory usage: 5.7+ KB


In [10]:
Urban_Bus_Operation_Status.isna().sum()

업체구분           0
노선번호           0
기점             0
종점             0
첫차             0
막차             0
배차간격           0
운행대수          12
운행거리(키로미터)    18
경유지            0
dtype: int64

In [11]:
Urban_Bus_Operation_Status['노선번호']

0            1번
1          1-1번
2            2번
3          2-1번
4            3번
        ...    
66       128-1번
67         221번
68        1004번
69    1004(심야)번
70         170번
Name: 노선번호, Length: 71, dtype: object

In [12]:
l=[]
line=[]
for i in Urban_Bus_Operation_Status['노선번호']:
    l.append(i.strip('번'))
    

In [13]:
Urban_Bus_Operation_Status['노선번호']=pd.Series(l)

In [14]:
Urban_Bus_Operation_Status

,업체구분,노선번호,기점,종점,첫차,막차,배차간격,운행대수,운행거리(키로미터),경유지
0,가야IBS,1,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
1,가야IBS,1-1,삼계기점,김해대학역,05:08/05:30,22:30/23:20,10분~15분,10.0,15.5,북부동사무소-푸르지오2차가락로-한일여고-안동4
2,동부교통,2,외동차고지,어방동,05:30/-,22:15/-,24~26분,5.0,27.6,가야고-금강병원-시청-안동6-인제대
3,동부교통,2-1,외동차고지,어방동,05:40/-,22:30/-,24~26분,5.0,27.0,가야고-세무서-시청-안동공단입구-인제대
4,김해버스,3,풍유동차고지,김해여객터미널(순환),06:10/-,21:20/-,고정배차,2.0,30.4,골든루트-마찰-흥동3-동아그린-가락로-교육청
...,...,...,...,...,...,...,...,...,...,...
66,부산버스,128-1,구산동,신라대학교,04:40/-,21:50/-,5~7분,26.0,54.0,구산동-인제대학교-신어중학교-구포역-신라대학교
67,부산버스,221,장유온천,하단전철역,06:00/-,20:00/-,-,NaN,NaN,장유온천-한림풀에버-수가마을-렛츠런파크입구-하단역
68,부산버스,1004,구산동,국제여객터미널,04:50/-,21:40/-,11~13분,NaN,NaN,구산동-금강병원-불암역-화인아파트-서면역-부산항국제여객터미널
69,부산버스,1004(심야),구산동,서구청,22:00/-,23:40/-,30분,NaN,NaN,구산동-금강병원-불암역-북부면허시험장-사상우체국-범일역-남포동


In [15]:
Urban_Bus_Operation_Status.rename(columns={'노선번호':'line_no'},inplace = True)

In [70]:
Urban_Bus_Operation_Status['line_no']

0            1
1          1-1
2            2
3          2-1
4            3
        ...   
66       128-1
67         221
68        1004
69    1004(심야)
70         170
Name: line_no, Length: 71, dtype: object

In [17]:
Urban_Line_Data=pd.merge(Bus_Routewise_WiFi_Usage_Data,Urban_Bus_Operation_Status,how='inner',on='line_no')

In [68]:
Urban_Line_Data

,line_no,std_ym,usage,업체구분,기점,종점,첫차,막차,배차간격,운행대수,운행거리(키로미터),경유지
0,1,202201,439.96,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
1,1,202202,399.48,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
2,1,202203,558.04,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
3,1,202204,627.30,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
4,1,202205,731.61,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,30.2,가야대-푸르지오2차-가락로-복음병원-인제대-한일여고
...,...,...,...,...,...,...,...,...,...,...,...,...
985,상동공영2,202302,15.86,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,53.5,보건소-교육청-호계로-복음병원-삼성초-인제대-나전-여차-매리-장척계곡-우계-활천고개...
986,상동공영2,202303,18.54,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,53.5,보건소-교육청-호계로-복음병원-삼성초-인제대-나전-여차-매리-장척계곡-우계-활천고개...
987,상동공영2,202304,17.40,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,53.5,보건소-교육청-호계로-복음병원-삼성초-인제대-나전-여차-매리-장척계곡-우계-활천고개...
988,상동공영2,202305,13.00,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,53.5,보건소-교육청-호계로-복음병원-삼성초-인제대-나전-여차-매리-장척계곡-우계-활천고개...


### Urban_Bus_Operation_Status 에서 경유지 부분 - 기준으로 다 나누기

In [71]:
bus_routes_df=pd.read_csv('버스 번호별 노선2.csv')
bus_routes_df

,Unnamed: 0,line_no,0,1,2,3,4,5,6,7,...,175,176,177,178,179,180,181,182,183,184
0,0,1,삼계기점,공원묘지입구,삼계서희,삼계동원로얄듀크,가야대학교,현대가야아이파크,삼계축협,매정공원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1-1,삼계기점,공원묘지입구,신명마을,감분마을,삼계푸르지오1차,가야대역,북부동행정복지센터,장신대역,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,10,진영역,설창,신용삼거리,신용마을,제암요양병원,용전마을,기영아파트,진영협성휴포레,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100,풍유동차고지,풍유동차고지앞,풍유동,청호아트빌,흥동3통,흥동1통,흥동2통,흥동4통,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,11,상룡,용담마을,신용삼거리,신용마을,제암요양병원,용전마을,기영아파트,진영협성휴포레,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,도시형1,한림면행정복지센터,한림정역,내오서,외오서,오서안골,내오서경로회관,금곡마을회관,정촌마을,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,86,도시형2,인제대역,메가마트,활천46통(수영),활천22통(전산),부원14통(쇠내),부원역,김해터미널(남문),봉황초등학교,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,87,도시형3,어병마을,모업,소업마을,퇴은,삼미,신기마을,진영역,효동마을,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,88,상동공영1,외동차고지(종점),이마트 신세계백화점,김해터미널(정문),봉황역,김해중부경찰서,부원역,김해세무서,김해천주교회,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df=pd.merge(Urban_Line_Data,bus_routes_df,how='left',on='line_no')
df

,line_no,std_ym,usage,업체구분,기점,종점,첫차,막차,배차간격,운행대수,...,175,176,177,178,179,180,181,182,183,184
0,1,202201,439.96,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,202202,399.48,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,202203,558.04,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,202204,627.30,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,202205,731.61,가야IBS,삼계기점,경남은행,05:00/05:20,22:16/23:15,10~18분,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,상동공영2,202302,15.86,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,상동공영2,202303,18.54,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987,상동공영2,202304,17.40,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,상동공영2,202305,13.00,김해버스,외동차고지,상동매리농협,08:00/06:35,21:00/18:20,고정배차,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2=pd.merge(

# 경전철
### Subway_Station_Status
- station_nm    경전철역 명
- addr    주소
- lon    경도
- lat    위도

### Subway_Station_Boarding_Alighting_Records
- gbn    분류
- date    영업일자
- station_nm    역 명
- TM_00    00시 인원
- TM_01    01시 인원  ...

In [19]:
Subway_Station_Status=pd.read_csv(input_path.joinpath('14.김해시경전철역현황.csv'))

Subway_Station_Boarding_Alighting_Records=pd.read_csv(input_path.joinpath('15.김해시경전철역별_승하차이력.csv'))

In [20]:
Subway_Station_Status

,station_nm,addr,lon,lat
0,가야대,경상남도 김해시 해반천로 211,128.865076,35.266724
1,공항,부산광역시 강서구 공항진입로 165,128.948588,35.171910
2,괘법 르네시떼,부산광역시 사상구 광장로 9,128.977685,35.163259
3,김해대학,경상남도 김해시 김해대로 2641,128.915615,35.228998
4,김해시청,경상남도 김해시 김해대로 2408,128.890415,35.227104
5,대사,부산광역시 강서구 낙동북로 82,128.938422,35.217654
6,대저,부산광역시 강서구 낙동북로 294,128.960566,35.213064
7,덕두,부산광역시 강서구 경전철로 10,128.954194,35.181998
8,등구,부산광역시 강서구 경전철로 194,128.963595,35.196147
9,박물관,경상남도 김해시 김해대로 2103,128.871868,35.240066


In [21]:
Subway_Station_Boarding_Alighting_Records

,gbn,date,station_nm,TM_00,TM_01,TM_02,TM_03,TM_04,TM_05,TM_06,...,TM_14,TM_15,TM_16,TM_17,TM_18,TM_19,TM_20,TM_21,TM_22,TM_23
0,승차,2021-01-01,사상,0,0,0,0,8,60,69,...,205,251,234,224,142,161,112,75,42,17
1,승차,2021-01-01,괘법 르네시떼,0,0,0,0,1,17,17,...,43,57,39,55,34,27,20,17,2,0
2,승차,2021-01-01,서부산 유통지구,0,0,0,0,1,0,7,...,14,15,21,27,16,6,2,8,6,0
3,승차,2021-01-01,공항,0,0,0,0,0,2,3,...,78,205,185,218,132,175,86,211,15,1
4,승차,2021-01-01,덕두,0,0,0,0,0,4,4,...,15,14,19,10,17,8,9,9,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25153,하차,2022-08-22,수로왕릉,19,0,0,0,0,9,51,...,111,111,171,231,310,168,100,103,58,40
25154,하차,2022-08-22,박물관,8,0,0,0,0,12,25,...,50,99,112,124,153,101,54,46,54,32
25155,하차,2022-08-22,연지공원,5,0,0,0,0,7,16,...,45,80,72,110,166,99,54,48,43,24
25156,하차,2022-08-22,장신대,13,0,0,0,0,7,38,...,97,178,150,217,305,172,107,75,80,40


In [22]:
Subway_Station=pd.merge(Subway_Station_Status,Subway_Station_Boarding_Alighting_Records,how='right',on='station_nm')

In [23]:
Subway_Station

,station_nm,addr,lon,lat,gbn,date,TM_00,TM_01,TM_02,TM_03,...,TM_14,TM_15,TM_16,TM_17,TM_18,TM_19,TM_20,TM_21,TM_22,TM_23
0,사상,부산광역시 사상구 광장로 83,128.985939,35.162300,승차,2021-01-01,0,0,0,0,...,205,251,234,224,142,161,112,75,42,17
1,사상,부산광역시 사상구 광장로 83,128.985939,35.162300,승차,2021-01-02,0,0,0,0,...,234,285,220,196,264,220,140,116,56,16
2,사상,부산광역시 사상구 광장로 83,128.985939,35.162300,승차,2021-01-03,0,0,0,0,...,228,229,215,312,294,270,163,118,46,12
3,사상,부산광역시 사상구 광장로 83,128.985939,35.162300,승차,2021-01-04,0,0,0,0,...,223,356,323,337,495,273,136,149,60,22
4,사상,부산광역시 사상구 광장로 83,128.985939,35.162300,승차,2021-01-05,0,0,0,0,...,271,300,315,336,486,261,132,130,61,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25153,가야대,경상남도 김해시 해반천로 211,128.865076,35.266724,하차,2022-08-18,17,0,0,0,...,68,116,186,224,306,186,121,92,94,80
25154,가야대,경상남도 김해시 해반천로 211,128.865076,35.266724,하차,2022-08-19,24,0,0,0,...,78,156,136,238,291,195,142,119,107,100
25155,가야대,경상남도 김해시 해반천로 211,128.865076,35.266724,하차,2022-08-20,21,0,0,0,...,82,75,93,127,125,101,120,135,87,58
25156,가야대,경상남도 김해시 해반천로 211,128.865076,35.266724,하차,2022-08-21,9,0,0,0,...,88,108,84,125,111,101,143,108,88,67


In [28]:
Subway_Station.date.value_counts()

2021-05-24    42
2022-05-16    42
2021-10-11    42
2021-07-05    42
2022-01-23    42
              ..
2021-08-18    42
2021-07-08    42
2022-04-03    42
2021-08-09    42
2022-08-09    42
Name: date, Length: 599, dtype: int64

> 시간에 따른 승하차인원이므로 시계열 분석 ㄱㄱ